# Compute the Daily temperature range (DTR)

In this example we compute the DTR as the monthly mean of the difference between the daily maximum temperature (TX) and minimum temperature (TN).

Firts, let's import PyOphidia and connect to server instance

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

Import Tasmin and Tasmax variables

In [ ]:
tasmax = cube.Cube(
    src_path='/data/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
    measure='tasmax',
    imp_dim='time',
    import_metadata='yes',
    imp_concept_level='d',
    ncores=4,
    description='Max Temperatures',
    hierarchy='oph_base|oph_base|oph_time',
    vocabulary='CF'
    )

In [ ]:
tasmin = cube.Cube(
    src_path='/data/tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
    measure='tasmin',
    imp_dim='time',
    import_metadata='yes',
    imp_concept_level='d',
    ncores=4,
    description='Min Temperatures',
    hierarchy='oph_base|oph_base|oph_time',
    vocabulary='CF'
    )

Compute daily DTR

In [ ]:
dailyDTR = tasmax.intercube(
    cube2=tasmin.pid,
    operation='sub',
    description="Daily DTR",
    ncores=4
    )

Compute monthly average of DTR

In [ ]:
monthlyDTR = dailyDTR.reduce2(
    dim='time',
    concept_level='M',
    operation='avg',
    description="Monthly DTR",
    ncores=4    
)

Extract the first month (Jan 2096)

In [ ]:
firstMonthDTR = monthlyDTR.subset(
    subset_dims='time',
    subset_type="coord",
    subset_filter='2096-01',
    description="Subset Monthly DTR",
    ncores = 4
)

data = firstMonthDTR.export_array()

Plot monthly DTR

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Oranges)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('DTR (deg K)')
plt.show()

Clear your workspace before running other notebooks

In [ ]:
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc', force='yes')
cube.Cube.deletecontainer(container='tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc', force='yes')